# Data Processing

In [ ]:
# Data
import pandas as pd
import numpy as np

# Models
from sklearn.neighbors import KNeighborsRegressor
import lightgbm as lgbm
from sklearn.linear_model import LinearRegression

# Pipelines/Processing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# Cross-Validation
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_validate, GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
import optuna
from optuna.integration import LightGBMPruningCallback
import optuna.integration.lightgbm as lgb

# Performance/Debugging
from sklearn import metrics
import time
from sklearn.metrics import log_loss
import warnings

In [ ]:
df = pd.read_csv(r'../input/autos/autos.csv', encoding='latin-1')
df = df.drop(['dateCrawled', 'lastSeen', 'postalCode', 'nrOfPictures', 'monthOfRegistration', 'dateCreated'], axis=1)
df

In [ ]:
for var in df.columns:
    na = sum(df[var].isna())/len(df)
    print(f'{var}: {100 * na:.3f}')

### Fixing Seller
Remove the cars being sold by businesss (only 3)

In [ ]:
df = df[df['seller'] != 'gewerblich'] # Remove the three offers made by businesses instead of people
df = df.drop('seller', axis = 1)

### Fixing offerType
Need to remove cars not being asked to offer on, but that are instead requests (Ex: requests to fix). Only 12 of these

In [ ]:
df = df[df['offerType'] != 'Gesuch']
df = df.drop('offerType', axis=1)

### Fixing Price
Some cars have values way above normal (Ex: 999999999 or 123123123). Remove inplausible numbers. It seems like 300000 is a good threshold for what is real after googling some of these cars. Remove 122 cars with exceedingly high price.

In [ ]:
print(sum(df['price'] > 300000))
df = df[df['price'] <= 300000]

Remove the cars with no price set (0). There are a lot of these, around 12k

In [ ]:
sum(df['price'] == 0)
df = df[df['price'] != 0]

In [ ]:
df[df['price'] < 500]['price'].hist()
sum(df['price'] < 500)
df = df[df['price'] >= 500]

### Fixing abtest
Not sure what this is but it doesnt seem to be too important. Probably can drop it.

In [ ]:
df['abtest'].value_counts()
df.boxplot('price', 'abtest')
df = df.drop('abtest', axis=1)

### Fixing Vehicle Type
Many cars are mislabelled. Will ignore those considered 'andere' or other as they have no classification.

Not sure what the difference between kombi and limousine are, joined them both into car category for now.

In [ ]:
print(df['vehicleType'].value_counts())
translations = {'kleinwagen': 'small car', # 100% sure
               'kombi': 'car', # Not sure what the difference between kombi and limousine are
               'cabrio': 'convertible', # 100% sure
               'limousine': 'car', # Not sure what the difference between kombi and limousine are
               'bus': 'van'}

df['vehicleType'] = df['vehicleType'].replace(translations)
df['vehicleType'].value_counts()


In [ ]:
df = df[df['vehicleType'] != 'andere']

### Fixing Year Of Registration
First remove completely impossible values (years in the future or before 1900)

In [ ]:
sum(df['yearOfRegistration'] > 2021)
sum(df['yearOfRegistration'] < 1900)
df = df[(df['yearOfRegistration'] < 2021) & (df['yearOfRegistration'] > 1900)]

In [ ]:
df['yearOfRegistration'].value_counts()

In [ ]:
df[df['yearOfRegistration'] == 1910] # These cars are definitely not from 1910

In [ ]:
df[df['yearOfRegistration'] == 1950] # These ones are right

In [ ]:
df = df[df['yearOfRegistration'] > 1950]

In [ ]:
df['yearOfRegistration'].hist()

### Fixing gearbox

In [ ]:
translations = {'manuell': 'manual',
               'automatik': 'automatic'}
df['gearbox'] = df['gearbox'].replace(translations)

boolean = {'automatik': True,
          'manuell': False}

df['automatic'] = df['gearbox'].replace(boolean)
df = df.drop('gearbox', axis = 1)
df['automatic'].value_counts()

### Fixing powerPS

No commercial car has more than 1000HP, and none have less than 5. Additionally, practically no cars have below 50hp, and very few above 400. 500 found to be good threshold.


In [ ]:
print(sum(df['powerPS'] > 1000))
df = df[df['powerPS'] < 500]
df['powerPS'].hist()

In [ ]:
df = df[df['powerPS'] > 50]

In [ ]:
df.sort_values('powerPS', ascending=False)

### Fixing Model
model: 5.513
Too many values, probably not worth trying to do anything with but IDK.


In [ ]:
df['model'].value_counts()

### Fixing kilometer

Wow, this actually is fine and no wrong values.

In [ ]:
df['kilometer'].hist()

### Fixing fuel type
Just need to translate and remove not categorized.


In [ ]:
translations = {'elektro': 'electric',
               'andere': 'other',
               'benzin': 'gas'}
df['fuelType'] = df['fuelType'].replace(translations)
df['fuelType'].value_counts()
df[df['fuelType'] != 'other']

### Fixing brand
Will remove any brand that doesnt have more than 100 listings (only 1 brand doesnt meet this)

In [ ]:
df['brand'].value_counts()
df = df[df['brand'] != 'trandere']

### Fixing damage

In [ ]:
df.notRepairedDamage
translations = {'nein': False,
               'ja': True}

df['damaged'] = df['notRepairedDamage'].replace(translations)
df['damaged'] = df['damaged'].fillna(False)

## Final Dataframe

In [ ]:
df = df.drop(['notRepairedDamage', 'model'], axis=1)

new_names = {'vehicleType': 'type',
            'yearOfRegistration': 'year',
            'powerPS': 'hp',
            'fuelType': 'fuel'}

df = df.rename(columns=new_names)


In [ ]:
for var in df.columns:
    na = sum(df[var].isna())/len(df)
    print(f'{var}: {100 * na:.3f}')

# Data Analysis
It appears year made and horsepower have the biggest effect on increasing price. Numbre of kilometers driven and whether damaged reduce price.

In [ ]:
df.corr()

In [ ]:
df = df.dropna()

In [ ]:
df_cat = df.copy()
df_cat['type'] = df['type'].astype('category')
df_cat['fuel'] = df['fuel'].astype('category')
df_cat['brand'] = df['brand'].astype('category')
df_cat['type']

# Preprocessing

Testing set is to be 20% of the data. Use seed 123 to split, and we will shuffle.

In [ ]:
num_vars = ['kilometer', 'year', 'hp']
cat_vars = ['type', 'fuel', 'brand', 'automatic', 'damaged']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[num_vars+cat_vars], df['price'],
                                                    test_size=0.2,
                                                    random_state=123,
                                                    shuffle=True)

In [ ]:
X_train

---
# Evaluation

### Model: Linear Regression

In [ ]:
linear_pipe = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('linear', LinearRegression())])

linear_pipe

In [ ]:
cross_val_score(linear_pipe, X_train, y_train, cv = 5)

---
## K-Nearest Neighbors 

Initial Imports

Initial Pipeline

In [ ]:
pipe = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', KNeighborsRegressor())])

pipe

### Cross Validation to determine whether to include the brand variable

In [ ]:
scores_all = cross_val_score(pipe, X_train, y_train, cv = 5)

--- 
# Trees

## Decision Tree


### Initial Pipeline

In [ ]:
from sklearn.tree import DecisionTreeRegressor

pipe = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', DecisionTreeRegressor(random_state = 1))])

pipe

### Do Not Run: Initial Performance



In [ ]:
scores_all = cross_val_score(pipe, X_train, y_train, cv = 5, scoring = 'neg_root_mean_squared_error')
np.mean(scores_all)

### Initial Performance Results:
-4140.518178545499

### Do Not Run: Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{"model__max_depth" : [1,5,10],
                "model__max_leaf_nodes":[None,10,50,100],
                "model__min_samples_leaf":[1,5,10]}]


gs = GridSearchCV(estimator= pipe,
                  param_grid=param_grid,
                  scoring = 'neg_root_mean_squared_error',
                  refit = True,
                  cv=5)

gs.fit(X_train, y_train)

print('Best RMSE: %.3f%%' % (gs.best_score_))
print('Best Params:', gs.best_params_)
print('Accuracy: %.3f%%' % (gs.best_estimator_.score(X_test, y_test)*100))

### Hyperparameter Tuning Results:
Best RMSE: -4014.652%
Best Params: {'model__max_depth': 10, 'model__max_leaf_nodes': None, 'model__min_samples_leaf': 5}
Accuracy: 77.921%

### Final Model
Using best parameter calculated with GridSearch

In [ ]:
pipe_tree = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', DecisionTreeRegressor(max_depth = 10, max_leaf_nodes = None, min_samples_leaf = 5, random_state = 1))])

pipe_tree

### Feature Importance

In [ ]:
pipe_tree.fit(X_train,y_train)
for importance, name in sorted(zip(pipe_tree.steps[1][1].feature_importances_, X_train.columns),reverse=True)[:10]:
    print (name, importance)

In [ ]:
import pylab as plt

features = [1,2,3,4,5,6,7,8]
importances = [.45292631965999713, .36813948422453335, .053961550813257306, .012371585304217203, .006461353694462267,  .0025208521747745066, .001924184773778391, .0006201678187062528]

LABELS = ["year", "hp", "kilometer", "type", "automatic", "damaged", "fuel", "brand"]

plt.bar(features, importances, align='center')
plt.xticks(features, LABELS)
plt.show()

### Final Prediction

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
pred = pipe_tree.predict(X_test)

print('RMSE on test data: ',  mean_squared_error(y_test, pred)**(0.5))
mape = mean_absolute_percentage_error(y_test, pred)*100
print('MAPE on test data: ',mape)

## Random Forests

### Initial Pipeline

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipe = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', RandomForestRegressor(random_state=1))])

pipe

### Do Not Run: Initial Baseline

In [ ]:
scores_all = cross_val_score(pipe, X_train, y_train, cv = 5, scoring = 'neg_root_mean_squared_error')
np.mean(scores_all)

### Initial Baseline Results: 
-3335.5370460504114

### Do Not Run: Hyperparamter Tuning

In [ ]:
param_grid = [ {      
                         'model__max_features': ['auto', 'sqrt'],
                         'model__min_samples_leaf': [2, 5],
                         'model__min_samples_split': [2, 5],
                    }]


gs = GridSearchCV(estimator= pipe,
                  param_grid=param_grid,
                  scoring = 'neg_root_mean_squared_error',
                  refit = True,
                  cv=5)

gs.fit(X_train, y_train)

print('Best RMSE: %.3f%%' % (gs.best_score_))
print('Best Params:', gs.best_params_)
print('Accuracy: %.3f%%' % (gs.best_estimator_.score(X_test, y_test)*100))

### Hyperparameter Tuning Results: 
Best RMSE: -3331.379%
Best Params: {'model__max_features': 'auto', 'model__min_samples_leaf': 2, 'model__min_samples_split': 5}
Accuracy: 84.763%

### Final Model

In [ ]:
pipe_forest = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', RandomForestRegressor(max_features = 'auto', min_samples_leaf = 2, min_samples_split = 5, random_state = 1))])

pipe_forest

### Do Not Run: Fitting and Feature Importance

In [ ]:
pipe_forest.fit(X_train,y_train)
for importance, name in sorted(zip(pipe_forest.steps[1][1].feature_importances_, X_train.columns),reverse=True)[:10]:
    print (name, importance)

### Fitting and Feature Importance Results


In [ ]:
import pylab as plt

features = [1,2,3,4,5,6,7,8]
importances = [.4284273329714948, .3451945656319133, .06310091808587716, .015230108605218376, .008297465775180667,  .004567383091394414, .004003910861843181, .001037417752473544]

LABELS = ["year", "hp", "kilometer", "type", "automatic", "damaged", "fuel", "brand"]

plt.bar(features, importances, align='center')
plt.xticks(features, LABELS)
plt.show()

### Final Prediction

In [ ]:
pred = pipe_forest.predict(X_test)

print('RMSE on test data: ',  mean_squared_error(y_test, pred)**(0.5))
mape = mean_absolute_percentage_error(y_test, pred)*100
print('MAPE on test data: ',mape)

---
# LightGBM

### Encoding categorical variables

In [ ]:
lgbm_pipe = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OneHotEncoder(sparse=True, drop='first'), cat_vars)])),
                      ('model', lgbm.LGBMRegressor(metric='rmse'))])

lgbm_pipe

In [ ]:
lgbm_scores = cross_val_score(lgbm_pipe, X_train, y_train, cv = 5, scoring='neg_root_mean_squared_error')
np.mean(-1 * lgbm_scores)

### Using raw categorical variables

In [ ]:
lgbm_pipe_cat = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OrdinalEncoder(dtype=int), cat_vars)])),
                      ('model', lgbm.LGBMRegressor(metric='rmse',
                                                  categorical_features=[3,4,5,6,7]))])

lgbm_pipe_cat

In [ ]:
lgbm_scores_cat = cross_val_score(lgbm_pipe_cat, X_train, y_train, cv = 5, scoring='neg_root_mean_squared_error')
np.mean(-1 * lgbm_scores_cat)

In [ ]:
lgbm_pipe_cat.get_params()

Slightly better performance using categorical features as is instead of encoding them. 

## Hyperparameter Tuning

In [ ]:
gridParams = {
    'model__learning_rate': [0.05, 0.1, 0.2],
    'model__n_estimators': [50,100,200],
    'model__num_leaves': [16, 31, 62], # large num_leaves helps improve accuracy but might lead to over-fitting
    'model__max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'model__random_state' : [500],
    'model__subsample' : [0.65, 0.7, 0.75],
    'model__reg_alpha' : [0,0.1],
    'model__reg_lambda' : [0,0.1],
    }

In [ ]:
search = GridSearchCV(lgbm_pipe_cat, gridParams, scoring='neg_root_mean_squared_error', verbose=2, n_jobs=-1)
search.fit(X_train, y_train)

In [ ]:
print(search.best_params_)
print(search.best_score_)

In [ ]:
best_params = {'learning_rate': 0.1,
               'max_bin': 510,
               'n_estimators': 200,
               'num_leaves': 62,
               'random_state': 500,
               'reg_alpha': 0,
               'reg_lambda': 0,
               'subsample': 0.65}

In [ ]:
lgbm_pipe_opt = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OrdinalEncoder(dtype=int), cat_vars)])),
                      ('model', lgbm.LGBMRegressor(metric='rmse',
                                                  categorical_features=[3,4,5,6,7],
                                                  **best_params))])

lgbm_pipe_opt

In [ ]:
lgbm_scores_opt = cross_val_score(lgbm_pipe_opt, X_train, y_train, cv = 5, scoring='neg_root_mean_squared_error')
np.mean(-1 * lgbm_scores_opt)

### Cross-Validation

In [ ]:
# model = lgbm.LGBMRegressor(**study.best_params)
# model.fit(X_train, y_train)
# preds = model.predict(X_test)

#### Optuna Auto-CV

In [ ]:
kf = KFold(shuffle=True, random_state=42)

for var in cat_vars:
    X_train[var] = OrdinalEncoder().fit_transform(X_train[var].values.reshape(-1,1))

params = {
        "objective": "regression",
        "metric": "l2",
        "verbosity": -1,
        "boosting_type": "gbdt",                
        "seed": 42
    }

X = np.array(X_train)    
y = np.array(y_train).flatten()

study_tuner = optuna.create_study(direction='minimize')
dtrain = lgb.Dataset(X, label=y)

# Suppress information only outputs - otherwise optuna is 
# quite verbose, which can be nice, but takes up a lot of space
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# Run optuna LightGBMTunerCV tuning of LightGBM with cross-validation
tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            categorical_feature=[3,4,5,6,7],
                            study=study_tuner,
                            early_stopping_rounds=50,
                            time_budget= 4 * 60 * 60, #19800, # Time budget of 5 hours, we will not really need it
                            seed = 42,
                            folds=kf,
                            )

tuner.run()

In [ ]:
# import json
# best_params = tuner.best_params
# with open('params.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(best_params))

In [ ]:
best_params = {"objective": "regression", "verbosity": -1, "boosting_type": "gbdt", "seed": 42, "feature_pre_filter": False, "lambda_l1": 2.9645157648969746e-06, "lambda_l2": 1.5726550639201562, "num_leaves": 256, "feature_fraction": 0.88, "bagging_fraction": 0.750496500444326, "bagging_freq": 1, "min_child_samples": 20}

In [ ]:
lgbm_pipe_optuna = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OrdinalEncoder(dtype=int), cat_vars)])),
                      ('model', lgbm.LGBMRegressor(metric='rmse',
                                                  categorical_features=[3,4,5,6,7],
                                                  **best_params))])

lgbm_pipe_optuna

In [ ]:
lgbm_scores_optuna = cross_val_score(lgbm_pipe_cat, X_train, y_train, cv = 5, scoring='neg_root_mean_squared_error')
np.mean(-1 * lgbm_scores_optuna)

In [ ]:
lgbm_pipe_mape = Pipeline([('features', ColumnTransformer([
    ('numeric', StandardScaler(), num_vars),
    ('categoric', OrdinalEncoder(dtype=int), cat_vars)])),
                      ('model', lgbm.LGBMRegressor(metric='mape',
                                                  categorical_features=[3,4,5,6,7],
                                                  **best_params))])

lgbm_pipe_mape

In [ ]:
#from sklearn.metrics import mean_absolute_percentage_error

lgbm_pipe_opt.fit(X_train, y_train)
preds_rmse = lgbm_pipe_opt.predict(X_test)
print(f'RMSE: {metrics.mean_squared_error(y_test, preds_rmse, squared=False)}')

lgbm_pipe_mape.fit(X_train, y_train)
#preds_mape = lgbm_pipe_mape.predict(X_test)
print(f'MAPE: {lgbm_pipe_opt.score(X_test, y_test)}')

In [ ]:
pd.DataFrame(lgbm_pipe_opt.steps[1][1].feature_importances_, X_train.columns).sort_values(by=0, ascending=False).plot.bar()

In [ ]:
pd.DataFrame(lgbm_pipe_mape.steps[1][1].feature_importances_, X_train.columns).sort_values(by=0, ascending=False).plot.bar()